In [687]:
# importings
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math
import seaborn as sns
%matplotlib notebook

In [688]:
#importing raw data
covid_raw = pd.read_csv('../Data/Raw/owid-covid-data.csv')

In [689]:
covid_raw.head(5)

,iso_code,continent,location,date,total_cases,new_cases,new_cases_smoothed,total_deaths,new_deaths,new_deaths_smoothed,...,male_smokers,handwashing_facilities,hospital_beds_per_thousand,life_expectancy,human_development_index,population,excess_mortality_cumulative_absolute,excess_mortality_cumulative,excess_mortality,excess_mortality_cumulative_per_million
0,AFG,Asia,Afghanistan,2020-01-05,NaN,0.0,NaN,NaN,0.0,NaN,...,NaN,37.746,0.5,64.83,0.511,41128772.0,NaN,NaN,NaN,NaN
1,AFG,Asia,Afghanistan,2020-01-06,NaN,0.0,NaN,NaN,0.0,NaN,...,NaN,37.746,0.5,64.83,0.511,41128772.0,NaN,NaN,NaN,NaN
2,AFG,Asia,Afghanistan,2020-01-07,NaN,0.0,NaN,NaN,0.0,NaN,...,NaN,37.746,0.5,64.83,0.511,41128772.0,NaN,NaN,NaN,NaN
3,AFG,Asia,Afghanistan,2020-01-08,NaN,0.0,NaN,NaN,0.0,NaN,...,NaN,37.746,0.5,64.83,0.511,41128772.0,NaN,NaN,NaN,NaN
4,AFG,Asia,Afghanistan,2020-01-09,NaN,0.0,NaN,NaN,0.0,NaN,...,NaN,37.746,0.5,64.83,0.511,41128772.0,NaN,NaN,NaN,NaN


In [690]:
print(covid_raw.columns)

Index(['iso_code', 'continent', 'location', 'date', 'total_cases', 'new_cases',
       'new_cases_smoothed', 'total_deaths', 'new_deaths',
       'new_deaths_smoothed', 'total_cases_per_million',
       'new_cases_per_million', 'new_cases_smoothed_per_million',
       'total_deaths_per_million', 'new_deaths_per_million',
       'new_deaths_smoothed_per_million', 'reproduction_rate', 'icu_patients',
       'icu_patients_per_million', 'hosp_patients',
       'hosp_patients_per_million', 'weekly_icu_admissions',
       'weekly_icu_admissions_per_million', 'weekly_hosp_admissions',
       'weekly_hosp_admissions_per_million', 'total_tests', 'new_tests',
       'total_tests_per_thousand', 'new_tests_per_thousand',
       'new_tests_smoothed', 'new_tests_smoothed_per_thousand',
       'positive_rate', 'tests_per_case', 'tests_units', 'total_vaccinations',
       'people_vaccinated', 'people_fully_vaccinated', 'total_boosters',
       'new_vaccinations', 'new_vaccinations_smoothed',
       't

In [691]:
covid_new = covid_raw.drop(['total_cases_per_million',
       'new_cases_per_million', 'new_cases_smoothed_per_million',
       'total_deaths_per_million', 'new_deaths_per_million',
       'new_deaths_smoothed_per_million',
       'icu_patients_per_million',
       'hosp_patients_per_million',
       'weekly_icu_admissions_per_million',
       'weekly_hosp_admissions_per_million',
       'total_tests_per_thousand', 'new_tests_per_thousand',
       'new_tests_smoothed', 'new_tests_smoothed_per_thousand',
       'total_vaccinations_per_hundred', 'people_vaccinated_per_hundred',
       'people_fully_vaccinated_per_hundred', 'total_boosters_per_hundred',
       'new_vaccinations_smoothed_per_million',
       'new_people_vaccinated_smoothed_per_hundred','excess_mortality_cumulative_per_million', 'total_tests',
        'new_tests',
        'positive_rate',
        'tests_per_case',
        'tests_units',
        'excess_mortality_cumulative_absolute',
        'excess_mortality_cumulative'], axis=1)
# For Test columns Because we already have cases columns for positve tests.

In [692]:
# We want to use data's until there are no new cases and deaths in a country (2023-07-02)
# USA didn't submit new cases since 2023-05-21
covid_new['date'] = pd.to_datetime(covid_new['date'])
covid_new = covid_new[covid_new['date'] < '2023-05-20']

In [693]:
print(covid_new.columns)

Index(['iso_code', 'continent', 'location', 'date', 'total_cases', 'new_cases',
       'new_cases_smoothed', 'total_deaths', 'new_deaths',
       'new_deaths_smoothed', 'reproduction_rate', 'icu_patients',
       'hosp_patients', 'weekly_icu_admissions', 'weekly_hosp_admissions',
       'total_vaccinations', 'people_vaccinated', 'people_fully_vaccinated',
       'total_boosters', 'new_vaccinations', 'new_vaccinations_smoothed',
       'new_people_vaccinated_smoothed', 'stringency_index',
       'population_density', 'median_age', 'aged_65_older', 'aged_70_older',
       'gdp_per_capita', 'extreme_poverty', 'cardiovasc_death_rate',
       'diabetes_prevalence', 'female_smokers', 'male_smokers',
       'handwashing_facilities', 'hospital_beds_per_thousand',
       'life_expectancy', 'human_development_index', 'population',
       'excess_mortality'],
      dtype='object')


In [694]:
# understandig missing datas:
nan_columns = covid_new.columns[covid_new.isna().any()].tolist()
nan_columns

['continent',
 'total_cases',
 'new_cases',
 'new_cases_smoothed',
 'total_deaths',
 'new_deaths',
 'new_deaths_smoothed',
 'reproduction_rate',
 'icu_patients',
 'hosp_patients',
 'weekly_icu_admissions',
 'weekly_hosp_admissions',
 'total_vaccinations',
 'people_vaccinated',
 'people_fully_vaccinated',
 'total_boosters',
 'new_vaccinations',
 'new_vaccinations_smoothed',
 'new_people_vaccinated_smoothed',
 'stringency_index',
 'population_density',
 'median_age',
 'aged_65_older',
 'aged_70_older',
 'gdp_per_capita',
 'extreme_poverty',
 'cardiovasc_death_rate',
 'diabetes_prevalence',
 'female_smokers',
 'male_smokers',
 'handwashing_facilities',
 'hospital_beds_per_thousand',
 'life_expectancy',
 'human_development_index',
 'excess_mortality']

now we are going to understand each column missing data and handle it:

In [695]:
# There are a lot of missing data for countries: England, Hong Kong, Macao, Northern Cyprus, Northern Ireland, Scotland, Taiwan, Wales
missing_data_locations = ['England', 'Hong Kong', 'Macao', 'Northern Cyprus', 'Northern Ireland', 'Scotland', 'Taiwan', 'Wales']
covid_new = covid_new[~covid_new['location'].isin(missing_data_locations) ]

In [696]:
# continent:
covid_new['continent'].isna().sum()

14772

<div dir=rtl >
<font  size=3>
در اینجا متوجه می‌شویم که اون‌هایی که ستون قاره مربوطه نال است، نام قاره در ستون مکان آمده است، حال اول چک کنیم که تعداد مبتلا‌های کشور‌های با خود قاره برابر است یا نه.
</font>
</div>

In [697]:
africa_datas = covid_new[covid_new['continent'] == 'Africa']
africa_continent_datas = covid_new[covid_new['location'] == 'Africa']
africa_datas[africa_datas['date'] == '2024-02-18']['total_cases'].sum() == africa_continent_datas[africa_continent_datas['date'] == '2024-02-18']['total_cases']

Series([], Name: total_cases, dtype: bool)

<div dir=rtl >
<font  size=3>
متوجه می‌شویم که داده‌هایی که ستون قاره آن ها خالی است تنها آمده است کشورها را جمع زده است. پس سطر‌های مربوط را پاک می‌کنیم.
</font>
</div>

In [698]:
covid_new = covid_new.drop(covid_new[covid_new['continent'].isna()].index, axis=0)

In [699]:
covid_new[covid_new['continent'].isna()].count

<bound method DataFrame.count of Empty DataFrame
Columns: [iso_code, continent, location, date, total_cases, new_cases, new_cases_smoothed, total_deaths, new_deaths, new_deaths_smoothed, reproduction_rate, icu_patients, hosp_patients, weekly_icu_admissions, weekly_hosp_admissions, total_vaccinations, people_vaccinated, people_fully_vaccinated, total_boosters, new_vaccinations, new_vaccinations_smoothed, new_people_vaccinated_smoothed, stringency_index, population_density, median_age, aged_65_older, aged_70_older, gdp_per_capita, extreme_poverty, cardiovasc_death_rate, diabetes_prevalence, female_smokers, male_smokers, handwashing_facilities, hospital_beds_per_thousand, life_expectancy, human_development_index, population, excess_mortality]
Index: []

[0 rows x 39 columns]>

In [700]:
# total_cases
covid_new[covid_new['total_cases'].isna()]

,iso_code,continent,location,date,total_cases,new_cases,new_cases_smoothed,total_deaths,new_deaths,new_deaths_smoothed,...,cardiovasc_death_rate,diabetes_prevalence,female_smokers,male_smokers,handwashing_facilities,hospital_beds_per_thousand,life_expectancy,human_development_index,population,excess_mortality
0,AFG,Asia,Afghanistan,2020-01-05,NaN,0.0,NaN,NaN,0.0,NaN,...,597.029,9.59,NaN,NaN,37.746,0.5,64.83,0.511,41128772.0,NaN
1,AFG,Asia,Afghanistan,2020-01-06,NaN,0.0,NaN,NaN,0.0,NaN,...,597.029,9.59,NaN,NaN,37.746,0.5,64.83,0.511,41128772.0,NaN
2,AFG,Asia,Afghanistan,2020-01-07,NaN,0.0,NaN,NaN,0.0,NaN,...,597.029,9.59,NaN,NaN,37.746,0.5,64.83,0.511,41128772.0,NaN
3,AFG,Asia,Afghanistan,2020-01-08,NaN,0.0,NaN,NaN,0.0,NaN,...,597.029,9.59,NaN,NaN,37.746,0.5,64.83,0.511,41128772.0,NaN
4,AFG,Asia,Afghanistan,2020-01-09,NaN,0.0,NaN,NaN,0.0,NaN,...,597.029,9.59,NaN,NaN,37.746,0.5,64.83,0.511,41128772.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
379176,ZWE,Africa,Zimbabwe,2020-03-17,NaN,0.0,0.0,5.0,0.0,0.0,...,307.846,1.82,1.6,30.7,36.791,1.7,61.49,0.571,16320539.0,NaN
379177,ZWE,Africa,Zimbabwe,2020-03-18,NaN,0.0,0.0,5.0,0.0,0.0,...,307.846,1.82,1.6,30.7,36.791,1.7,61.49,0.571,16320539.0,NaN
379178,ZWE,Africa,Zimbabwe,2020-03-19,NaN,0.0,0.0,5.0,0.0,0.0,...,307.846,1.82,1.6,30.7,36.791,1.7,61.49,0.571,16320539.0,NaN
379179,ZWE,Africa,Zimbabwe,2020-03-20,NaN,0.0,0.0,5.0,0.0,0.0,...,307.846,1.82,1.6,30.7,36.791,1.7,61.49,0.571,16320539.0,NaN


In [701]:
# we can see that when there is no cases the total_cases is NaN so we replace the NaN with 0:
covid_new['total_cases'] = covid_new['total_cases'].fillna(0)

In [702]:
covid_new[covid_new['total_cases'].isna()]

,iso_code,continent,location,date,total_cases,new_cases,new_cases_smoothed,total_deaths,new_deaths,new_deaths_smoothed,...,cardiovasc_death_rate,diabetes_prevalence,female_smokers,male_smokers,handwashing_facilities,hospital_beds_per_thousand,life_expectancy,human_development_index,population,excess_mortality


In [703]:
# new_cases
covid_new[covid_new['new_cases'].isna()]

,iso_code,continent,location,date,total_cases,new_cases,new_cases_smoothed,total_deaths,new_deaths,new_deaths_smoothed,...,cardiovasc_death_rate,diabetes_prevalence,female_smokers,male_smokers,handwashing_facilities,hospital_beds_per_thousand,life_expectancy,human_development_index,population,excess_mortality
13554,ARG,South America,Argentina,2020-01-01,0.0,NaN,NaN,NaN,NaN,NaN,...,191.032,5.50,16.2,27.7,NaN,5.00,76.67,0.845,45510324.0,NaN
13555,ARG,South America,Argentina,2020-01-02,0.0,NaN,NaN,NaN,NaN,NaN,...,191.032,5.50,16.2,27.7,NaN,5.00,76.67,0.845,45510324.0,NaN
13556,ARG,South America,Argentina,2020-01-03,0.0,NaN,NaN,NaN,NaN,NaN,...,191.032,5.50,16.2,27.7,NaN,5.00,76.67,0.845,45510324.0,NaN
13557,ARG,South America,Argentina,2020-01-04,0.0,NaN,NaN,NaN,NaN,NaN,...,191.032,5.50,16.2,27.7,NaN,5.00,76.67,0.845,45510324.0,NaN
34033,BLZ,North America,Belize,2022-04-10,57287.0,NaN,NaN,664.0,8.0,1.143,...,176.957,17.11,NaN,NaN,90.083,1.30,74.62,0.716,405285.0,NaN
92277,ECU,South America,Ecuador,2020-09-13,116451.0,NaN,NaN,10864.0,4140.0,591.429,...,140.448,5.55,2.0,12.3,80.635,1.50,77.01,0.759,18001002.0,38.93
98707,GNQ,Africa,Equatorial Guinea,2022-03-20,15800.0,NaN,NaN,183.0,0.0,0.000,...,202.812,7.78,NaN,NaN,24.640,2.10,58.74,0.592,1674916.0,NaN
135722,GTM,North America,Guatemala,2020-04-05,50.0,NaN,NaN,1.0,0.0,0.000,...,155.898,10.18,NaN,NaN,76.665,0.60,74.30,0.663,17843914.0,-20.96
199308,MWI,Africa,Malawi,2022-11-06,88009.0,NaN,NaN,2685.0,2.0,0.286,...,227.349,3.94,4.4,24.7,8.704,1.30,64.26,0.483,20405318.0,NaN
201392,MDV,Asia,Maldives,2020-03-29,16.0,NaN,NaN,NaN,0.0,0.000,...,164.905,9.19,2.1,55.0,95.803,NaN,78.92,0.740,523798.0,NaN


In [704]:
# It seems that these remaining NaN data for new cases are 0 at the beginning of the pandemic
covid_new['new_cases'] = covid_new['new_cases'].fillna(0)

In [705]:
# making sure that we don't have any problem in total_cases for new_cases we filled.
for country_name in covid_new['location'].unique():
    index_values = covid_new[(covid_new['new_cases'] == 0) & (covid_new['location'] == country_name)]['total_cases']
    index_zero = covid_new[(covid_new['new_cases'] == 0) & (covid_new['location'] == country_name)].index
    if not index_zero.empty:
        index_zero = index_zero.tolist()
        index_zero.pop()
        next_index = [i + 1 for i in index_zero]
        next_cases_value = covid_new.loc[next_index, 'total_cases']
        difference = next_cases_value - index_values
        difference = difference.tolist()
        print(f'{country_name} : {list(filter(lambda x: x != 0 and x is not None and not math.isnan(x), difference))}')
    else:
        print(f'{country_name} : No index with zero new cases found')

Afghanistan : []
Albania : []
Algeria : []
American Samoa : []
Andorra : []
Angola : []
Anguilla : []
Antigua and Barbuda : []
Argentina : []
Armenia : []
Aruba : []
Australia : []
Austria : []
Azerbaijan : []
Bahamas : []
Bahrain : []
Bangladesh : []
Barbados : []
Belarus : []
Belgium : []
Belize : []
Benin : []
Bermuda : []
Bhutan : []
Bolivia : []
Bonaire Sint Eustatius and Saba : []
Bosnia and Herzegovina : []
Botswana : []
Brazil : []
British Virgin Islands : []
Brunei : []
Bulgaria : []
Burkina Faso : []
Burundi : []
Cambodia : []
Cameroon : []
Canada : []
Cape Verde : []
Cayman Islands : []
Central African Republic : []
Chad : []
Chile : []
China : []
Colombia : []
Comoros : []
Congo : []
Cook Islands : []
Costa Rica : []
Cote d'Ivoire : []
Croatia : []
Cuba : []
Curacao : []
Cyprus : []
Czechia : []
Democratic Republic of Congo : []
Denmark : []
Djibouti : []
Dominica : []
Dominican Republic : []
Ecuador : []
Egypt : []
El Salvador : []
Equatorial Guinea : []
Eritrea : []
Eston

In [706]:
# new_cases_smoothed
covid_new[covid_new['new_cases_smoothed'].isna()]

,iso_code,continent,location,date,total_cases,new_cases,new_cases_smoothed,total_deaths,new_deaths,new_deaths_smoothed,...,cardiovasc_death_rate,diabetes_prevalence,female_smokers,male_smokers,handwashing_facilities,hospital_beds_per_thousand,life_expectancy,human_development_index,population,excess_mortality
0,AFG,Asia,Afghanistan,2020-01-05,0.0,0.0,NaN,NaN,0.0,NaN,...,597.029,9.59,NaN,NaN,37.746,0.5,64.83,0.511,41128772.0,NaN
1,AFG,Asia,Afghanistan,2020-01-06,0.0,0.0,NaN,NaN,0.0,NaN,...,597.029,9.59,NaN,NaN,37.746,0.5,64.83,0.511,41128772.0,NaN
2,AFG,Asia,Afghanistan,2020-01-07,0.0,0.0,NaN,NaN,0.0,NaN,...,597.029,9.59,NaN,NaN,37.746,0.5,64.83,0.511,41128772.0,NaN
3,AFG,Asia,Afghanistan,2020-01-08,0.0,0.0,NaN,NaN,0.0,NaN,...,597.029,9.59,NaN,NaN,37.746,0.5,64.83,0.511,41128772.0,NaN
4,AFG,Asia,Afghanistan,2020-01-09,0.0,0.0,NaN,NaN,0.0,NaN,...,597.029,9.59,NaN,NaN,37.746,0.5,64.83,0.511,41128772.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
379104,ZWE,Africa,Zimbabwe,2020-01-05,0.0,0.0,NaN,NaN,0.0,NaN,...,307.846,1.82,1.6,30.7,36.791,1.7,61.49,0.571,16320539.0,NaN
379105,ZWE,Africa,Zimbabwe,2020-01-06,0.0,0.0,NaN,NaN,0.0,NaN,...,307.846,1.82,1.6,30.7,36.791,1.7,61.49,0.571,16320539.0,NaN
379106,ZWE,Africa,Zimbabwe,2020-01-07,0.0,0.0,NaN,NaN,0.0,NaN,...,307.846,1.82,1.6,30.7,36.791,1.7,61.49,0.571,16320539.0,NaN
379107,ZWE,Africa,Zimbabwe,2020-01-08,0.0,0.0,NaN,NaN,0.0,NaN,...,307.846,1.82,1.6,30.7,36.791,1.7,61.49,0.571,16320539.0,NaN


In [707]:
# the same thing as new cases
covid_new['new_cases_smoothed'] = covid_new['new_cases_smoothed'].fillna(0)

In [708]:
# Creating a new column : total_cases_smoothed
covid_new['total_cases_smoothed'] = covid_new['new_cases_smoothed'].cumsum()

In [709]:
# total_deaths
covid_new[covid_new['total_deaths'].isna()]

,iso_code,continent,location,date,total_cases,new_cases,new_cases_smoothed,total_deaths,new_deaths,new_deaths_smoothed,...,diabetes_prevalence,female_smokers,male_smokers,handwashing_facilities,hospital_beds_per_thousand,life_expectancy,human_development_index,population,excess_mortality,total_cases_smoothed
0,AFG,Asia,Afghanistan,2020-01-05,0.0,0.0,0.0,NaN,0.0,NaN,...,9.59,NaN,NaN,37.746,0.5,64.83,0.511,41128772.0,NaN,0.000000e+00
1,AFG,Asia,Afghanistan,2020-01-06,0.0,0.0,0.0,NaN,0.0,NaN,...,9.59,NaN,NaN,37.746,0.5,64.83,0.511,41128772.0,NaN,0.000000e+00
2,AFG,Asia,Afghanistan,2020-01-07,0.0,0.0,0.0,NaN,0.0,NaN,...,9.59,NaN,NaN,37.746,0.5,64.83,0.511,41128772.0,NaN,0.000000e+00
3,AFG,Asia,Afghanistan,2020-01-08,0.0,0.0,0.0,NaN,0.0,NaN,...,9.59,NaN,NaN,37.746,0.5,64.83,0.511,41128772.0,NaN,0.000000e+00
4,AFG,Asia,Afghanistan,2020-01-09,0.0,0.0,0.0,NaN,0.0,NaN,...,9.59,NaN,NaN,37.746,0.5,64.83,0.511,41128772.0,NaN,0.000000e+00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
379162,ZWE,Africa,Zimbabwe,2020-03-03,0.0,0.0,0.0,NaN,0.0,0.0,...,1.82,1.6,30.7,36.791,1.7,61.49,0.571,16320539.0,NaN,7.652433e+08
379163,ZWE,Africa,Zimbabwe,2020-03-04,0.0,0.0,0.0,NaN,0.0,0.0,...,1.82,1.6,30.7,36.791,1.7,61.49,0.571,16320539.0,NaN,7.652433e+08
379164,ZWE,Africa,Zimbabwe,2020-03-05,0.0,0.0,0.0,NaN,0.0,0.0,...,1.82,1.6,30.7,36.791,1.7,61.49,0.571,16320539.0,NaN,7.652433e+08
379165,ZWE,Africa,Zimbabwe,2020-03-06,0.0,0.0,0.0,NaN,0.0,0.0,...,1.82,1.6,30.7,36.791,1.7,61.49,0.571,16320539.0,NaN,7.652433e+08


In [710]:
covid_new['total_deaths'] = covid_new['total_deaths'].fillna(0)

In [711]:
covid_new[covid_new['total_cases'].isna()]

,iso_code,continent,location,date,total_cases,new_cases,new_cases_smoothed,total_deaths,new_deaths,new_deaths_smoothed,...,diabetes_prevalence,female_smokers,male_smokers,handwashing_facilities,hospital_beds_per_thousand,life_expectancy,human_development_index,population,excess_mortality,total_cases_smoothed


In [712]:
# new_deaths
covid_new[covid_new['new_deaths'].isna()]

,iso_code,continent,location,date,total_cases,new_cases,new_cases_smoothed,total_deaths,new_deaths,new_deaths_smoothed,...,diabetes_prevalence,female_smokers,male_smokers,handwashing_facilities,hospital_beds_per_thousand,life_expectancy,human_development_index,population,excess_mortality,total_cases_smoothed
13554,ARG,South America,Argentina,2020-01-01,0.0,0.0,0.000,0.0,NaN,NaN,...,5.50,16.2,27.7,NaN,5.00,76.67,0.845,45510324.0,NaN,9.993719e+05
13555,ARG,South America,Argentina,2020-01-02,0.0,0.0,0.000,0.0,NaN,NaN,...,5.50,16.2,27.7,NaN,5.00,76.67,0.845,45510324.0,NaN,9.993719e+05
13556,ARG,South America,Argentina,2020-01-03,0.0,0.0,0.000,0.0,NaN,NaN,...,5.50,16.2,27.7,NaN,5.00,76.67,0.845,45510324.0,NaN,9.993719e+05
13557,ARG,South America,Argentina,2020-01-04,0.0,0.0,0.000,0.0,NaN,NaN,...,5.50,16.2,27.7,NaN,5.00,76.67,0.845,45510324.0,NaN,9.993719e+05
41388,BES,North America,Bonaire Sint Eustatius and Saba,2021-10-17,2306.0,102.0,14.571,19.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,77.79,NaN,27052.0,NaN,3.981674e+07
58223,CAN,North America,Canada,2022-06-19,3915721.0,23312.0,3330.286,41017.0,NaN,NaN,...,7.37,12.0,16.6,NaN,2.50,82.43,0.929,38454328.0,5.54,8.390407e+07
66057,CHL,South America,Chile,2023-04-02,5264330.0,14253.0,2036.143,61050.0,NaN,NaN,...,8.46,34.2,41.5,NaN,2.11,80.18,0.851,19603736.0,6.37,9.004861e+07
132920,GLP,North America,Guadeloupe,2020-11-01,7742.0,268.0,38.286,126.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,82.14,NaN,395762.0,18.21,2.985051e+08
213379,MEX,North America,Mexico,2020-01-01,0.0,0.0,0.000,0.0,NaN,NaN,...,13.06,6.9,21.4,87.847,1.38,75.05,0.779,127504120.0,NaN,4.469132e+08
213380,MEX,North America,Mexico,2020-01-02,0.0,0.0,0.000,0.0,NaN,NaN,...,13.06,6.9,21.4,87.847,1.38,75.05,0.779,127504120.0,NaN,4.469132e+08


In [713]:
# It seems that these remaining NaN data for new death are 0 at the beginning of the pandemic
covid_new['new_deaths'] = covid_new['new_deaths'].fillna(0)

In [714]:
# making sure that we have problem in total_deaths for new_deaths we filled.
for country_name in covid_new['location'].unique():
    index_values = covid_new[(covid_new['new_deaths'] == 0) & (covid_new['location'] == country_name)]['total_deaths']
    index_zero = covid_new[(covid_new['new_deaths'] == 0) & (covid_new['location'] == country_name)].index
    if not index_zero.empty:
        index_zero = index_zero.tolist()
        index_zero.pop()
        next_index = [i + 1 for i in index_zero]
        next_cases_value = covid_new.loc[next_index, 'total_deaths']
        difference = next_cases_value - index_values
        difference = difference.tolist()
        print(f'{country_name} : {list(filter(lambda x: x != 0 and x is not None and not math.isnan(x), difference))}')
    else:
        print(f'{country_name} : No index with zero new cases found')

Afghanistan : []
Albania : []
Algeria : []
American Samoa : []
Andorra : []
Angola : []
Anguilla : []
Antigua and Barbuda : []
Argentina : []
Armenia : []
Aruba : []
Australia : []
Austria : []
Azerbaijan : []
Bahamas : []
Bahrain : []
Bangladesh : []
Barbados : []
Belarus : []
Belgium : []
Belize : []
Benin : []
Bermuda : []
Bhutan : []
Bolivia : []
Bonaire Sint Eustatius and Saba : []
Bosnia and Herzegovina : []
Botswana : []
Brazil : []
British Virgin Islands : []
Brunei : []
Bulgaria : []
Burkina Faso : []
Burundi : []
Cambodia : []
Cameroon : []
Canada : []
Cape Verde : []
Cayman Islands : []
Central African Republic : []
Chad : []
Chile : []
China : []
Colombia : []
Comoros : []
Congo : []
Cook Islands : []
Costa Rica : []
Cote d'Ivoire : []
Croatia : []
Cuba : []
Curacao : []
Cyprus : []
Czechia : []
Democratic Republic of Congo : []
Denmark : []
Djibouti : []
Dominica : []
Dominican Republic : []
Ecuador : []
Egypt : []
El Salvador : []
Equatorial Guinea : []
Eritrea : []
Eston

In [715]:
covid_new[covid_new['new_deaths_smoothed'].isna()]

,iso_code,continent,location,date,total_cases,new_cases,new_cases_smoothed,total_deaths,new_deaths,new_deaths_smoothed,...,diabetes_prevalence,female_smokers,male_smokers,handwashing_facilities,hospital_beds_per_thousand,life_expectancy,human_development_index,population,excess_mortality,total_cases_smoothed
0,AFG,Asia,Afghanistan,2020-01-05,0.0,0.0,0.0,0.0,0.0,NaN,...,9.59,NaN,NaN,37.746,0.5,64.83,0.511,41128772.0,NaN,0.000000e+00
1,AFG,Asia,Afghanistan,2020-01-06,0.0,0.0,0.0,0.0,0.0,NaN,...,9.59,NaN,NaN,37.746,0.5,64.83,0.511,41128772.0,NaN,0.000000e+00
2,AFG,Asia,Afghanistan,2020-01-07,0.0,0.0,0.0,0.0,0.0,NaN,...,9.59,NaN,NaN,37.746,0.5,64.83,0.511,41128772.0,NaN,0.000000e+00
3,AFG,Asia,Afghanistan,2020-01-08,0.0,0.0,0.0,0.0,0.0,NaN,...,9.59,NaN,NaN,37.746,0.5,64.83,0.511,41128772.0,NaN,0.000000e+00
4,AFG,Asia,Afghanistan,2020-01-09,0.0,0.0,0.0,0.0,0.0,NaN,...,9.59,NaN,NaN,37.746,0.5,64.83,0.511,41128772.0,NaN,0.000000e+00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
379104,ZWE,Africa,Zimbabwe,2020-01-05,0.0,0.0,0.0,0.0,0.0,NaN,...,1.82,1.6,30.7,36.791,1.7,61.49,0.571,16320539.0,NaN,7.652433e+08
379105,ZWE,Africa,Zimbabwe,2020-01-06,0.0,0.0,0.0,0.0,0.0,NaN,...,1.82,1.6,30.7,36.791,1.7,61.49,0.571,16320539.0,NaN,7.652433e+08
379106,ZWE,Africa,Zimbabwe,2020-01-07,0.0,0.0,0.0,0.0,0.0,NaN,...,1.82,1.6,30.7,36.791,1.7,61.49,0.571,16320539.0,NaN,7.652433e+08
379107,ZWE,Africa,Zimbabwe,2020-01-08,0.0,0.0,0.0,0.0,0.0,NaN,...,1.82,1.6,30.7,36.791,1.7,61.49,0.571,16320539.0,NaN,7.652433e+08


In [716]:
covid_new['new_deaths_smoothed'] = covid_new['new_deaths_smoothed'].fillna(0)

In [717]:
# Creating a new column : total_deaths_smoothed
covid_new['total_deaths_smoothed'] = covid_new['new_deaths_smoothed'].cumsum()

In [718]:
# reproduction_rate
covid_new['reproduction_rate'].isna().sum()/len(covid_new['reproduction_rate'])

0.3656930404354588

In [719]:
# As we can see about 37 percent of data's missing, so we have to choose how we can fill the null cells
# First we plot box plot for each country:

sns.set(style="whitegrid")
fig, axs = plt.subplots(47, 5, figsize=(15, 200), sharex=True, sharey=True)
axs = axs.flatten()

for i, country_name in enumerate(covid_new['location'].unique()):
    country_data = covid_new[covid_new['location'] == country_name]
    print(f"{country_name} : {country_data['reproduction_rate'].isna().sum()/len(country_data)}")
    sns.boxplot(y=country_data['reproduction_rate'].dropna(), ax=axs[i])
    axs[i].set_title(f'Reproduction Rate: {country_name}', fontsize=6)
    axs[i].tick_params(axis='x', labelrotation=45)
    axs[i].set_xlabel('')
    axs[i].set_ylabel('')

# Remove extra subplots
for j in range(len(covid_new['location'].unique()), len(axs)):
    fig.delaxes(axs[j])

plt.tight_layout()
plt.show()


<IPython.core.display.Javascript object>

Afghanistan : 0.17952883834281072
Albania : 0.17546709991876522


Algeria : 0.17384240454914704
American Samoa : 1.0
Andorra : 0.17465475223395613
Angola : 0.239642567018684
Anguilla : 1.0
Antigua and Barbuda : 0.3281884646628757
Argentina : 0.1757085020242915
Armenia : 0.17221770917952883
Aruba : 1.0
Australia : 0.16490658001624695
Austria : 0.16490658001624695
Azerbaijan : 0.17790414297319251
Bahamas : 0.22502030869212022
Bahrain : 0.16490658001624695
Bangladesh : 0.1868399675060926
Barbados : 0.264825345247766
Belarus : 0.18115353371242893
Belgium : 0.16165718927701056
Belize : 0.2859463850528026
Benin : 0.21202274573517466
Bermuda : 1.0
Bhutan : 0.2802599512591389
Bolivia : 0.18196588139723802
Bonaire Sint Eustatius and Saba : 1.0
Bosnia and Herzegovina : 0.17465475223395613
Botswana : 0.2550771730300569
Brazil : 0.16734362307067424
British Virgin Islands : 1.0
Brunei : 0.17627944760357434
Bulgaria : 0.17303005686433795
Burkina Faso : 0.17627944760357434
Burundi : 0.2445166531275386
Cambodia : 0.1803411860276198
Cameroon : 0.1803411860276198
Cana

<div dir=rtl >
<font  size=3>
زمانی که به نمودار‌ها نگاه می‌کنیم متوجه می شویم که برای آن‌هایی که داده وجود دارد، قرار دادن میانه ایده بدی نیست، همچنین برای آن‌هایی که داده وجود ندارد مقدار منفی یک خواهیم گذاشت.
</font>
</div>

In [720]:
for country_name in covid_new['location'].unique():
    country_data = covid_new[covid_new['location'] == country_name]
    if (country_data['reproduction_rate'].isna().sum() / len(country_data)):
        covid_new.loc[covid_new['location'] == country_name, 'reproduction_rate'] = -1
    else:
        median = country_data['reproduction_rate'].median()
        covid_new.loc[covid_new['location'] == country_name,
                      'reproduction_rate'] = covid_new.loc[covid_new['location'] == country_name,
                                                           'reproduction_rate'].fillna(median)
# Check if there is any null left:
covid_new['reproduction_rate'].isna().sum()

0

In [721]:
# icu_patients
# We have to check icu_patients for each country individually:
sns.set(style="whitegrid")
fig, axs = plt.subplots(47, 5, figsize=(15, 200), sharex=True, sharey=True)
axs = axs.flatten()
for i, country_name in enumerate(covid_new['location'].unique()):
    country_data = covid_new[covid_new['location'] == country_name]
    ratio_of_null_data = country_data['icu_patients'].isna().sum()/len(country_data)
    print(f"{country_name} : {ratio_of_null_data}")
    sns.boxplot(y=country_data['icu_patients'].dropna(), ax=axs[i])
    axs[i].set_title(f'icu_patients: {country_name}', fontsize=10)
    axs[i].tick_params(axis='x', labelrotation=45)
    axs[i].set_xlabel('')
    axs[i].set_ylabel('')

for j in range(len(covid_new['location'].unique()), len(axs)):
    fig.delaxes(axs[j])

plt.tight_layout()
plt.show()
       

<IPython.core.display.Javascript object>

Afghanistan : 1.0
Albania : 1.0
Algeria : 0.330625507717303
American Samoa : 1.0
Andorra : 1.0
Angola : 1.0
Anguilla : 1.0
Antigua and Barbuda : 1.0
Argentina : 0.4761133603238866
Armenia : 1.0
Aruba : 1.0
Australia : 0.06986190089358245
Austria : 0.07067424857839155
Azerbaijan : 1.0
Bahamas : 1.0
Bahrain : 1.0
Bangladesh : 1.0
Barbados : 1.0
Belarus : 1.0
Belgium : 0.04955320877335499
Belize : 1.0
Benin : 1.0
Bermuda : 1.0
Bhutan : 1.0
Bolivia : 0.5743298131600325
Bonaire Sint Eustatius and Saba : 1.0
Bosnia and Herzegovina : 1.0
Botswana : 1.0
Brazil : 1.0
British Virgin Islands : 1.0
Brunei : 1.0
Bulgaria : 0.8708367181153533
Burkina Faso : 1.0
Burundi : 1.0
Cambodia : 1.0
Cameroon : 1.0
Canada : 0.07067424857839155
Cape Verde : 1.0
Cayman Islands : 1.0
Central African Republic : 1.0
Chad : 1.0
Chile : 0.07067424857839155
China : 1.0
Colombia : 1.0
Comoros : 1.0
Congo : 1.0
Cook Islands : 1.0
Costa Rica : 1.0
Cote d'Ivoire : 1.0
Croatia : 1.0
Cuba : 1.0
Curacao : 1.0
Cyprus : 0.3354

<div dir=rtl >
<font  size=3>
برای کشور‌هایی که داده وجود ندارد مقدار -۱ می‌گذاریم و برای بقیه از میانه استفاده می‌کنیم.
</font>
</div>

In [722]:
for country_name in covid_new['location'].unique():
    country_data = covid_new[covid_new['location'] == country_name]
    if (country_data['icu_patients'].isna().sum() / len(country_data)):
        covid_new.loc[covid_new['location'] == country_name, 'icu_patients'] = -1
    else:
        median_reproduction_rate = country_data['icu_patients'].median()
        covid_new.loc[covid_new['location'] == country_name,
                      'icu_patients'] = covid_new.loc[covid_new['location'] == country_name,
                                                           'icu_patients'].fillna(median_reproduction_rate)
# Check if there is any null left:
covid_new['icu_patients'].isna().sum()

0

In [723]:
# hosp_patients
# We do exacly like above:

sns.set(style="whitegrid")
fig, axs = plt.subplots(47, 5, figsize=(15, 200), sharex=True, sharey=True)
axs = axs.flatten()
for i, country_name in enumerate(covid_new['location'].unique()):
    country_data = covid_new[covid_new['location'] == country_name]
    ratio_of_null_data = country_data['hosp_patients'].isna().sum()/len(country_data)
    print(f"{country_name} : {ratio_of_null_data}")
    sns.boxplot(y=country_data['hosp_patients'].dropna(), ax=axs[i])
    axs[i].set_title(f'hosp_patients: {country_name}', fontsize=10)
    axs[i].tick_params(axis='x', labelrotation=45)
    axs[i].set_xlabel('')
    axs[i].set_ylabel('')

for j in range(len(covid_new['location'].unique()), len(axs)):
    fig.delaxes(axs[j])

plt.tight_layout()
plt.show()

<IPython.core.display.Javascript object>

Afghanistan : 1.0
Albania : 1.0
Algeria : 1.0
American Samoa : 1.0
Andorra : 1.0
Angola : 1.0
Anguilla : 1.0
Antigua and Barbuda : 1.0
Argentina : 1.0
Armenia : 1.0
Aruba : 1.0
Australia : 0.06986190089358245
Austria : 0.07067424857839155
Azerbaijan : 1.0
Bahamas : 1.0
Bahrain : 1.0
Bangladesh : 1.0
Barbados : 1.0
Belarus : 1.0
Belgium : 0.04955320877335499
Belize : 1.0
Benin : 1.0
Bermuda : 1.0
Bhutan : 1.0
Bolivia : 0.5743298131600325
Bonaire Sint Eustatius and Saba : 1.0
Bosnia and Herzegovina : 1.0
Botswana : 1.0
Brazil : 1.0
British Virgin Islands : 1.0
Brunei : 1.0
Bulgaria : 0.8700243704305443
Burkina Faso : 1.0
Burundi : 1.0
Cambodia : 1.0
Cameroon : 1.0
Canada : 0.07067424857839155
Cape Verde : 1.0
Cayman Islands : 1.0
Central African Republic : 1.0
Chad : 1.0
Chile : 1.0
China : 1.0
Colombia : 1.0
Comoros : 1.0
Congo : 1.0
Cook Islands : 1.0
Costa Rica : 1.0
Cote d'Ivoire : 1.0
Croatia : 0.3021933387489846
Cuba : 1.0
Curacao : 1.0
Cyprus : 0.3363119415109667
Czechia : 0.05361

In [724]:
for country_name in covid_new['location'].unique():
    country_data = covid_new[covid_new['location'] == country_name]
    if (country_data['hosp_patients'].isna().sum() / len(country_data)):
        covid_new.loc[covid_new['location'] == country_name, 'hosp_patients'] = -1
    else:
        median = country_data['hosp_patients'].median()
        covid_new.loc[covid_new['location'] == country_name,
                      'hosp_patients'] = covid_new.loc[covid_new['location'] == country_name,
                                                           'hosp_patients'].fillna(median)
# Check if there is any null left:
covid_new['hosp_patients'].isna().sum()

0

In [725]:
# weekly_icu_admissions

sns.set(style="whitegrid")
fig, axs = plt.subplots(47, 5, figsize=(15, 200), sharex=True, sharey=True)
axs = axs.flatten()
for i, country_name in enumerate(covid_new['location'].unique()):
    country_data = covid_new[covid_new['location'] == country_name]
    ratio_of_null_data = country_data['weekly_icu_admissions'].isna().sum()/len(country_data)
    print(f"{country_name} : {ratio_of_null_data}")
    sns.boxplot(y=country_data['weekly_icu_admissions'].dropna(), ax=axs[i])
    axs[i].set_title(f'weekly_icu_admissions: {country_name}', fontsize=6)
    axs[i].tick_params(axis='x', labelrotation=45)
    axs[i].set_xlabel('')
    axs[i].set_ylabel('')

for j in range(len(covid_new['location'].unique()), len(axs)):
    fig.delaxes(axs[j])

plt.tight_layout()
plt.show()

<IPython.core.display.Javascript object>

Afghanistan : 1.0
Albania : 1.0
Algeria : 1.0
American Samoa : 1.0
Andorra : 1.0
Angola : 1.0
Anguilla : 1.0
Antigua and Barbuda : 1.0
Argentina : 1.0
Armenia : 1.0
Aruba : 1.0
Australia : 1.0
Austria : 1.0
Azerbaijan : 1.0
Bahamas : 1.0
Bahrain : 1.0
Bangladesh : 1.0
Barbados : 1.0
Belarus : 1.0
Belgium : 1.0
Belize : 1.0
Benin : 1.0
Bermuda : 1.0
Bhutan : 1.0
Bolivia : 1.0
Bonaire Sint Eustatius and Saba : 1.0
Bosnia and Herzegovina : 1.0
Botswana : 1.0
Brazil : 1.0
British Virgin Islands : 1.0
Brunei : 1.0
Bulgaria : 1.0
Burkina Faso : 1.0
Burundi : 1.0
Cambodia : 1.0
Cameroon : 1.0
Canada : 1.0
Cape Verde : 1.0
Cayman Islands : 1.0
Central African Republic : 1.0
Chad : 1.0
Chile : 0.0934199837530463
China : 1.0
Colombia : 1.0
Comoros : 1.0
Congo : 1.0
Cook Islands : 1.0
Costa Rica : 1.0
Cote d'Ivoire : 1.0
Croatia : 1.0
Cuba : 1.0
Curacao : 1.0
Cyprus : 0.9114541023558083
Czechia : 0.050365556458164096
Democratic Republic of Congo : 1.0
Denmark : 1.0
Djibouti : 1.0
Dominica : 1.0
D

In [726]:
for country_name in covid_new['location'].unique():
    country_data = covid_new[covid_new['location'] == country_name]
    if (country_data['weekly_icu_admissions'].isna().sum() / len(country_data)):
        covid_new.loc[covid_new['location'] == country_name, 'weekly_icu_admissions'] = -1
    else:
        median_reproduction_rate = country_data['weekly_icu_admissions'].median()
        covid_new.loc[covid_new['location'] == country_name,
                      'weekly_icu_admissions'] = covid_new.loc[covid_new['location'] == country_name,
                                                           'weekly_icu_admissions'].fillna(median_reproduction_rate)
# Check if there is any null left:
covid_new['weekly_icu_admissions'].isna().sum()

0

In [727]:
# weekly_hosp_admissions

sns.set(style="whitegrid")
fig, axs = plt.subplots(47, 5, figsize=(15, 200), sharex=True, sharey=True)
axs = axs.flatten()
for i, country_name in enumerate(covid_new['location'].unique()):
    country_data = covid_new[covid_new['location'] == country_name]
    ratio_of_null_data = country_data['weekly_hosp_admissions'].isna().sum()/len(country_data)
    print(f"{country_name} : {ratio_of_null_data}")
    sns.boxplot(y=country_data['weekly_hosp_admissions'].dropna(), ax=axs[i])
    axs[i].set_title(f'weekly_hosp_admissions: {country_name}', fontsize=6)
    axs[i].tick_params(axis='x', labelrotation=45)
    axs[i].set_xlabel('')
    axs[i].set_ylabel('')

for j in range(len(covid_new['location'].unique()), len(axs)):
    fig.delaxes(axs[j])

plt.tight_layout()
plt.show()

<IPython.core.display.Javascript object>

Afghanistan : 1.0
Albania : 1.0
Algeria : 1.0
American Samoa : 1.0
Andorra : 1.0
Angola : 1.0
Anguilla : 1.0
Antigua and Barbuda : 1.0
Argentina : 1.0
Armenia : 1.0
Aruba : 1.0
Australia : 1.0
Austria : 1.0
Azerbaijan : 1.0
Bahamas : 1.0
Bahrain : 1.0
Bangladesh : 1.0
Barbados : 1.0
Belarus : 1.0
Belgium : 0.054427294882209584
Belize : 1.0
Benin : 1.0
Bermuda : 1.0
Bhutan : 1.0
Bolivia : 1.0
Bonaire Sint Eustatius and Saba : 1.0
Bosnia and Herzegovina : 1.0
Botswana : 1.0
Brazil : 1.0
British Virgin Islands : 1.0
Brunei : 1.0
Bulgaria : 1.0
Burkina Faso : 1.0
Burundi : 1.0
Cambodia : 1.0
Cameroon : 1.0
Canada : 1.0
Cape Verde : 1.0
Cayman Islands : 1.0
Central African Republic : 1.0
Chad : 1.0
Chile : 0.10398050365556458
China : 1.0
Colombia : 1.0
Comoros : 1.0
Congo : 1.0
Cook Islands : 1.0
Costa Rica : 1.0
Cote d'Ivoire : 1.0
Croatia : 0.90089358245329
Cuba : 1.0
Curacao : 1.0
Cyprus : 0.8748984565393989
Czechia : 0.050365556458164096
Democratic Republic of Congo : 1.0
Denmark : 0.05

In [728]:
for country_name in covid_new['location'].unique():
    country_data = covid_new[covid_new['location'] == country_name]
    if (country_data['weekly_hosp_admissions'].isna().sum() / len(country_data)):
        covid_new.loc[covid_new['location'] == country_name, 'weekly_hosp_admissions'] = -1
    else:
        median = country_data['weekly_hosp_admissions'].median()
        covid_new.loc[covid_new['location'] == country_name,
                      'weekly_hosp_admissions'] = covid_new.loc[covid_new['location'] == country_name,
                                                           'weekly_hosp_admissions'].fillna(median)
# Check if there is any null left:
covid_new['weekly_hosp_admissions'].isna().sum()

0

In [729]:
# stringency_index
sns.set(style="whitegrid")
fig, axs = plt.subplots(47, 5, figsize=(15, 200), sharex=True, sharey=True)
axs = axs.flatten()
for i, country_name in enumerate(covid_new['location'].unique()):
    country_data = covid_new[covid_new['location'] == country_name]
    ratio_of_null_data = country_data['stringency_index'].isna().sum()/len(country_data)
    print(f"{country_name} : {ratio_of_null_data}")
    sns.boxplot(y=country_data['stringency_index'].dropna(), ax=axs[i])
    axs[i].set_title(f'stringency_index: {country_name}', fontsize=6)
    axs[i].tick_params(axis='x', labelrotation=45)
    axs[i].set_xlabel('')
    axs[i].set_ylabel('')

for j in range(len(covid_new['location'].unique()), len(axs)):
    fig.delaxes(axs[j])

plt.tight_layout()
plt.show()

<IPython.core.display.Javascript object>

Afghanistan : 0.11291632818846466
Albania : 0.11291632818846466
Algeria : 0.11291632818846466
American Samoa : 1.0
Andorra : 0.11291632818846466
Angola : 0.11291632818846466
Anguilla : 1.0
Antigua and Barbuda : 1.0
Argentina : 0.1125506072874494
Armenia : 1.0
Aruba : 0.11291632818846466
Australia : 0.11291632818846466
Austria : 0.11291632818846466
Azerbaijan : 0.11291632818846466
Bahamas : 0.11291632818846466
Bahrain : 0.11291632818846466
Bangladesh : 0.11291632818846466
Barbados : 0.11291632818846466
Belarus : 0.11291632818846466
Belgium : 0.11291632818846466
Belize : 0.11291632818846466
Benin : 0.11291632818846466
Bermuda : 0.11291632818846466
Bhutan : 0.11291632818846466
Bolivia : 0.11291632818846466
Bonaire Sint Eustatius and Saba : 1.0
Bosnia and Herzegovina : 0.11291632818846466
Botswana : 0.11291632818846466
Brazil : 0.11291632818846466
British Virgin Islands : 1.0
Brunei : 0.11291632818846466
Bulgaria : 0.11291632818846466
Burkina Faso : 0.11291632818846466
Burundi : 0.11291632

<div dir=rtl >
<font  size=3>
این شاخص که نشان می‌دهد هر کشور چقدر قرنطینه را جدی گرفته است. به نظر بهتر است از مد برای پر کردن داده‌های خالی استفاده کنیم، چرا که معمولا این کشور‌ها معمولا این قوانین را پیاده سازی کرده
</font>
</div>

In [730]:
for country_name in covid_new['location'].unique():
    country_data = covid_new[covid_new['location'] == country_name]
    if (country_data['stringency_index'].isna().sum() / len(country_data)):
        covid_new.loc[covid_new['location'] == country_name, 'stringency_index'] = -1
    else:
        mode = country_data['stringency_index'].mode().tolist()[0]
        covid_new.loc[covid_new['location'] == country_name,
                      'stringency_index'] = covid_new.loc[covid_new['location'] == country_name,
                                                           'stringency_index'].fillna(mode)
# Check if there is any null left:
covid_new['stringency_index'].isna().sum()

0

In [731]:
# population_density
for country_name in covid_new['location'].unique():
    country_data = covid_new[covid_new['location'] == country_name]
    ratio_of_null_data = country_data['population_density'].isna().sum()/len(country_data)
    if ratio_of_null_data != 0:
        print(f"{country_name} : {ratio_of_null_data}")

Anguilla : 1.0
Bonaire Sint Eustatius and Saba : 1.0


Cook Islands : 1.0
Falkland Islands : 1.0
French Guiana : 1.0
Guadeloupe : 1.0
Guernsey : 1.0
Jersey : 1.0
Martinique : 1.0
Mayotte : 1.0
Montserrat : 1.0
Niue : 1.0
Pitcairn : 1.0
Reunion : 1.0
Saint Barthelemy : 1.0
Saint Helena : 1.0
Saint Pierre and Miquelon : 1.0
South Sudan : 1.0
Syria : 1.0
Tokelau : 1.0
Vatican : 1.0
Wallis and Futuna : 1.0
Western Sahara : 1.0


In [732]:
nan_countries = covid_new.groupby('location')['population_density'].transform('count') == 0
covid_new.loc[nan_countries, 'population_density'] = -1
covid_new['population_density'].isna().sum()

0

In [733]:
# median_age
for country_name in covid_new['location'].unique():
    country_data = covid_new[covid_new['location'] == country_name]
    ratio_of_null_data = country_data['median_age'].isna().sum()/len(country_data)
    if ratio_of_null_data != 0:
        print(f"{country_name} : {ratio_of_null_data}")

American Samoa : 1.0
Andorra : 1.0
Anguilla : 1.0
Bermuda : 1.0
Bonaire Sint Eustatius and Saba : 1.0
British Virgin Islands : 1.0
Cayman Islands : 1.0


Cook Islands : 1.0
Dominica : 1.0
Faeroe Islands : 1.0
Falkland Islands : 1.0
Gibraltar : 1.0
Greenland : 1.0
Guernsey : 1.0
Isle of Man : 1.0
Jersey : 1.0
Kosovo : 1.0
Liechtenstein : 1.0
Marshall Islands : 1.0
Monaco : 1.0
Montserrat : 1.0
Nauru : 1.0
Niue : 1.0
Northern Mariana Islands : 1.0
Palau : 1.0
Pitcairn : 1.0
Saint Barthelemy : 1.0
Saint Helena : 1.0
Saint Kitts and Nevis : 1.0
Saint Martin (French part) : 1.0
Saint Pierre and Miquelon : 1.0
San Marino : 1.0
Sint Maarten (Dutch part) : 1.0
Tokelau : 1.0
Turks and Caicos Islands : 1.0
Tuvalu : 1.0
Vatican : 1.0
Wallis and Futuna : 1.0


In [734]:
nan_countries = covid_new.groupby('location')['median_age'].transform('count') == 0
covid_new.loc[nan_countries, 'median_age'] = -1
covid_new['median_age'].isna().sum()

0

In [735]:
# aged_65_older
for country_name in covid_new['location'].unique():
    country_data = covid_new[covid_new['location'] == country_name]
    ratio_of_null_data = country_data['aged_65_older'].isna().sum()/len(country_data)
    if ratio_of_null_data != 0:
        print(f"{country_name} : {ratio_of_null_data}")

American Samoa : 1.0
Andorra : 1.0
Anguilla : 1.0
Bermuda : 1.0
Bonaire Sint Eustatius and Saba : 1.0
British Virgin Islands : 1.0
Cayman Islands : 1.0
Cook Islands : 1.0
Dominica : 1.0


Faeroe Islands : 1.0
Falkland Islands : 1.0
French Guiana : 1.0
Gibraltar : 1.0
Greenland : 1.0
Guadeloupe : 1.0
Guernsey : 1.0
Isle of Man : 1.0
Jersey : 1.0
Kosovo : 1.0
Liechtenstein : 1.0
Marshall Islands : 1.0
Martinique : 1.0
Mayotte : 1.0
Monaco : 1.0
Montserrat : 1.0
Nauru : 1.0
Niue : 1.0
Northern Mariana Islands : 1.0
Palau : 1.0
Pitcairn : 1.0
Reunion : 1.0
Saint Barthelemy : 1.0
Saint Helena : 1.0
Saint Kitts and Nevis : 1.0
Saint Martin (French part) : 1.0
Saint Pierre and Miquelon : 1.0
San Marino : 1.0
Sint Maarten (Dutch part) : 1.0
Syria : 1.0
Tokelau : 1.0
Turks and Caicos Islands : 1.0
Tuvalu : 1.0
Vatican : 1.0
Wallis and Futuna : 1.0
Western Sahara : 1.0


In [736]:
nan_countries = covid_new.groupby('location')['aged_65_older'].transform('count') == 0
covid_new.loc[nan_countries, 'aged_65_older'] = -1
covid_new['aged_65_older'].isna().sum()

0

In [737]:
# aged_70_older
for country_name in covid_new['location'].unique():
    country_data = covid_new[covid_new['location'] == country_name]
    ratio_of_null_data = country_data['aged_70_older'].isna().sum()/len(country_data)
    if ratio_of_null_data != 0:
        print(f"{country_name} : {ratio_of_null_data}")

American Samoa : 1.0
Andorra : 1.0
Anguilla : 1.0
Bermuda : 1.0
Bonaire Sint Eustatius and Saba : 1.0
British Virgin Islands : 1.0
Cayman Islands : 1.0


Cook Islands : 1.0
Dominica : 1.0
Faeroe Islands : 1.0
Falkland Islands : 1.0
Gibraltar : 1.0
Greenland : 1.0
Guadeloupe : 1.0
Guernsey : 1.0
Isle of Man : 1.0
Jersey : 1.0
Kosovo : 1.0
Liechtenstein : 1.0
Marshall Islands : 1.0
Monaco : 1.0
Montserrat : 1.0
Nauru : 1.0
Niue : 1.0
Northern Mariana Islands : 1.0
Palau : 1.0
Pitcairn : 1.0
Saint Barthelemy : 1.0
Saint Helena : 1.0
Saint Kitts and Nevis : 1.0
Saint Martin (French part) : 1.0
Saint Pierre and Miquelon : 1.0
San Marino : 1.0
Serbia : 1.0
Sint Maarten (Dutch part) : 1.0
Tokelau : 1.0
Turks and Caicos Islands : 1.0
Tuvalu : 1.0
Vatican : 1.0
Wallis and Futuna : 1.0


In [738]:
nan_countries = covid_new.groupby('location')['aged_70_older'].transform('count') == 0
covid_new.loc[nan_countries, 'aged_70_older'] = -1
covid_new['aged_70_older'].isna().sum()

0

In [739]:
# gdp_per_capita
for country_name in covid_new['location'].unique():
    country_data = covid_new[covid_new['location'] == country_name]
    ratio_of_null_data = country_data['gdp_per_capita'].isna().sum()/len(country_data)
    if ratio_of_null_data != 0:
        print(f"{country_name} : {ratio_of_null_data}")

American Samoa : 1.0
Andorra : 1.0
Anguilla : 1.0
Bonaire Sint Eustatius and Saba : 1.0
British Virgin Islands : 1.0


Cook Islands : 1.0
Cuba : 1.0
Curacao : 1.0
Faeroe Islands : 1.0
Falkland Islands : 1.0
French Guiana : 1.0
French Polynesia : 1.0
Gibraltar : 1.0
Greenland : 1.0
Guadeloupe : 1.0
Guam : 1.0
Guernsey : 1.0
Isle of Man : 1.0
Jersey : 1.0
Liechtenstein : 1.0
Martinique : 1.0
Mayotte : 1.0
Monaco : 1.0
Montserrat : 1.0
New Caledonia : 1.0
Niue : 1.0
North Korea : 1.0
Northern Mariana Islands : 1.0
Pitcairn : 1.0
Reunion : 1.0
Saint Barthelemy : 1.0
Saint Helena : 1.0
Saint Martin (French part) : 1.0
Saint Pierre and Miquelon : 1.0
Somalia : 1.0
Syria : 1.0
Tokelau : 1.0
Turks and Caicos Islands : 1.0
United States Virgin Islands : 1.0
Vatican : 1.0
Wallis and Futuna : 1.0
Western Sahara : 1.0


In [740]:
nan_countries = covid_new.groupby('location')['gdp_per_capita'].transform('count') == 0
covid_new.loc[nan_countries, 'gdp_per_capita'] = -1
covid_new['gdp_per_capita'].isna().sum()

0

In [741]:
# extreme_poverty
for country_name in covid_new['location'].unique():
    country_data = covid_new[covid_new['location'] == country_name]
    ratio_of_null_data = country_data['extreme_poverty'].isna().sum()/len(country_data)
    if ratio_of_null_data != 0:
        print(f"{country_name} : {ratio_of_null_data}")

Afghanistan : 1.0
American Samoa : 1.0
Andorra : 1.0
Angola : 1.0
Anguilla : 1.0
Antigua and Barbuda : 1.0
Aruba : 1.0
Azerbaijan : 1.0
Bahamas : 1.0
Bahrain : 1.0
Barbados : 1.0
Belarus : 1.0
Belize : 1.0
Bermuda : 1.0
Bonaire Sint Eustatius and Saba : 1.0
Botswana : 1.0
British Virgin Islands : 1.0
Brunei : 1.0
Cambodia : 1.0
Cape Verde : 1.0
Cayman Islands : 1.0
Central African Republic : 1.0
Cook Islands : 1.0
Cuba : 1.0
Curacao : 1.0
Cyprus : 1.0
Czechia : 1.0
Dominica : 1.0


Equatorial Guinea : 1.0
Eritrea : 1.0
Eswatini : 1.0
Faeroe Islands : 1.0
Falkland Islands : 1.0
Finland : 1.0
France : 1.0
French Guiana : 1.0
French Polynesia : 1.0
Germany : 1.0
Gibraltar : 1.0
Greenland : 1.0
Grenada : 1.0
Guadeloupe : 1.0
Guam : 1.0
Guernsey : 1.0
Guyana : 1.0
Isle of Man : 1.0
Jamaica : 1.0
Japan : 1.0
Jersey : 1.0
Kiribati : 1.0
Kuwait : 1.0
Lebanon : 1.0
Libya : 1.0
Liechtenstein : 1.0
Maldives : 1.0
Mali : 1.0
Marshall Islands : 1.0
Martinique : 1.0
Mayotte : 1.0
Monaco : 1.0
Montserrat : 1.0
Nauru : 1.0
Netherlands : 1.0
New Caledonia : 1.0
New Zealand : 1.0
Nigeria : 1.0
Niue : 1.0
North Korea : 1.0
Northern Mariana Islands : 1.0
Oman : 1.0
Palau : 1.0
Papua New Guinea : 1.0
Philippines : 1.0
Pitcairn : 1.0
Poland : 1.0
Puerto Rico : 1.0
Qatar : 1.0
Reunion : 1.0
Saint Barthelemy : 1.0
Saint Helena : 1.0
Saint Kitts and Nevis : 1.0
Saint Lucia : 1.0
Saint Martin (French part) : 1.0
Saint Pierre and Miquelon : 1.0
Saint Vincent and the Grenadines : 1.0
Samoa 

In [742]:
nan_countries = covid_new.groupby('location')['extreme_poverty'].transform('count') == 0
covid_new.loc[nan_countries, 'extreme_poverty'] = -1
covid_new['extreme_poverty'].isna().sum()

0

In [743]:
# cardiovasc_death_rate
for country_name in covid_new['location'].unique():
    country_data = covid_new[covid_new['location'] == country_name]
    ratio_of_null_data = country_data['cardiovasc_death_rate'].isna().sum()/len(country_data)
    if ratio_of_null_data != 0:
        print(f"{country_name} : {ratio_of_null_data}")

Anguilla : 1.0
Aruba : 1.0
Bonaire Sint Eustatius and Saba : 1.0
British Virgin Islands : 1.0
Cayman Islands : 1.0
Cook Islands : 1.0
Curacao : 1.0
Faeroe Islands : 1.0
Falkland Islands : 1.0


French Guiana : 1.0
French Polynesia : 1.0
Gibraltar : 1.0
Guadeloupe : 1.0
Guernsey : 1.0
Isle of Man : 1.0
Jersey : 1.0
Kosovo : 1.0
Liechtenstein : 1.0
Martinique : 1.0
Mayotte : 1.0
Monaco : 1.0
Montserrat : 1.0
Nauru : 1.0
New Caledonia : 1.0
Niue : 1.0
Palau : 1.0
Pitcairn : 1.0
Reunion : 1.0
Saint Barthelemy : 1.0
Saint Helena : 1.0
Saint Kitts and Nevis : 1.0
Saint Martin (French part) : 1.0
Saint Pierre and Miquelon : 1.0
San Marino : 1.0
Sint Maarten (Dutch part) : 1.0
Tokelau : 1.0
Turks and Caicos Islands : 1.0
Tuvalu : 1.0
Vatican : 1.0
Wallis and Futuna : 1.0
Western Sahara : 1.0


In [744]:
nan_countries = covid_new.groupby('location')['cardiovasc_death_rate'].transform('count') == 0
covid_new.loc[nan_countries, 'cardiovasc_death_rate'] = -1
covid_new['cardiovasc_death_rate'].isna().sum()

0

In [745]:
# diabetes_prevalence
for country_name in covid_new['location'].unique():
    country_data = covid_new[covid_new['location'] == country_name]
    ratio_of_null_data = country_data['diabetes_prevalence'].isna().sum()/len(country_data)
    if ratio_of_null_data != 0:
        print(f"{country_name} : {ratio_of_null_data}")

American Samoa : 1.0
Anguilla : 1.0
Bonaire Sint Eustatius and Saba : 1.0
Cook Islands : 1.0
Faeroe Islands : 1.0
Falkland Islands : 1.0
French Guiana : 1.0
Gibraltar : 1.0
Guadeloupe : 1.0
Guernsey : 1.0


Isle of Man : 1.0
Jersey : 1.0
Kosovo : 1.0
Martinique : 1.0
Mayotte : 1.0
Montserrat : 1.0
Niue : 1.0
Northern Mariana Islands : 1.0
Pitcairn : 1.0
Reunion : 1.0
Saint Barthelemy : 1.0
Saint Helena : 1.0
Saint Martin (French part) : 1.0
Saint Pierre and Miquelon : 1.0
Sint Maarten (Dutch part) : 1.0
Syria : 1.0
Tokelau : 1.0
Turks and Caicos Islands : 1.0
Vatican : 1.0
Wallis and Futuna : 1.0
Western Sahara : 1.0


In [746]:
nan_countries = covid_new.groupby('location')['diabetes_prevalence'].transform('count') == 0
covid_new.loc[nan_countries, 'diabetes_prevalence'] = -1
covid_new['diabetes_prevalence'].isna().sum()

0

In [747]:
# female_smokers
for country_name in covid_new['location'].unique():
    country_data = covid_new[covid_new['location'] == country_name]
    ratio_of_null_data = country_data['female_smokers'].isna().sum()/len(country_data)
    if ratio_of_null_data != 0:
        print(f"{country_name} : {ratio_of_null_data}")

Afghanistan : 1.0
American Samoa : 1.0
Angola : 1.0
Anguilla : 1.0
Antigua and Barbuda : 1.0
Aruba : 1.0
Belize : 1.0
Bermuda : 1.0
Bhutan : 1.0
Bolivia : 1.0
Bonaire Sint Eustatius and Saba : 1.0
British Virgin Islands : 1.0
Burundi : 1.0
Cameroon : 1.0
Cayman Islands : 1.0
Central African Republic : 1.0
Chad : 1.0
Cook Islands : 1.0
Cote d'Ivoire : 1.0
Curacao : 1.0
Democratic Republic of Congo : 1.0
Dominica : 1.0
Equatorial Guinea : 1.0
Faeroe Islands : 1.0
Falkland Islands : 1.0
French Guiana : 1.0
French Polynesia : 1.0
Gabon : 1.0
Gibraltar : 1.0
Greenland : 1.0
Grenada : 1.0
Guadeloupe : 1.0
Guam : 1.0
Guatemala : 1.0
Guernsey : 1.0


Guinea : 1.0
Guinea-Bissau : 1.0
Guyana : 1.0
Iraq : 1.0
Isle of Man : 1.0
Jersey : 1.0
Jordan : 1.0
Kosovo : 1.0
Libya : 1.0
Liechtenstein : 1.0
Madagascar : 1.0
Marshall Islands : 1.0
Martinique : 1.0
Mauritania : 1.0
Mayotte : 1.0
Micronesia (country) : 1.0
Monaco : 1.0
Montserrat : 1.0
New Caledonia : 1.0
Nicaragua : 1.0
Niue : 1.0
North Korea : 1.0
North Macedonia : 1.0
Northern Mariana Islands : 1.0
Palestine : 1.0
Pitcairn : 1.0
Puerto Rico : 1.0
Reunion : 1.0
Saint Barthelemy : 1.0
Saint Helena : 1.0
Saint Kitts and Nevis : 1.0
Saint Lucia : 1.0
Saint Martin (French part) : 1.0
Saint Pierre and Miquelon : 1.0
Saint Vincent and the Grenadines : 1.0
San Marino : 1.0
Sao Tome and Principe : 1.0
Sint Maarten (Dutch part) : 1.0
Solomon Islands : 1.0
Somalia : 1.0
South Sudan : 1.0
Sudan : 1.0
Syria : 1.0
Tajikistan : 1.0
Tokelau : 1.0
Trinidad and Tobago : 1.0
Turkmenistan : 1.0
Turks and Caicos Islands : 1.0
Tuvalu : 1.0
United States Virgin Islands : 1.0
Vatican : 1.0
Venezuela : 

In [748]:
nan_countries = covid_new.groupby('location')['female_smokers'].transform('count') == 0
covid_new.loc[nan_countries, 'female_smokers'] = -1
covid_new['female_smokers'].isna().sum()

0

In [749]:
# male_smokers
for country_name in covid_new['location'].unique():
    country_data = covid_new[covid_new['location'] == country_name]
    ratio_of_null_data = country_data['male_smokers'].isna().sum()/len(country_data)
    if ratio_of_null_data != 0:
        print(f"{country_name} : {ratio_of_null_data}")

Afghanistan : 1.0
American Samoa : 1.0
Angola : 1.0
Anguilla : 1.0
Antigua and Barbuda : 1.0
Aruba : 1.0
Belize : 1.0
Bermuda : 1.0
Bhutan : 1.0
Bolivia : 1.0
Bonaire Sint Eustatius and Saba : 1.0
British Virgin Islands : 1.0
Burundi : 1.0
Cameroon : 1.0
Cayman Islands : 1.0
Central African Republic : 1.0
Chad : 1.0
Cook Islands : 1.0
Cote d'Ivoire : 1.0


Curacao : 1.0
Democratic Republic of Congo : 1.0
Dominica : 1.0
Equatorial Guinea : 1.0
Faeroe Islands : 1.0
Falkland Islands : 1.0
French Guiana : 1.0
French Polynesia : 1.0
Gabon : 1.0
Gibraltar : 1.0
Greenland : 1.0
Grenada : 1.0
Guadeloupe : 1.0
Guam : 1.0
Guatemala : 1.0
Guernsey : 1.0
Guinea : 1.0
Guinea-Bissau : 1.0
Guyana : 1.0
Honduras : 1.0
Iraq : 1.0
Isle of Man : 1.0
Jersey : 1.0
Jordan : 1.0
Kosovo : 1.0
Libya : 1.0
Liechtenstein : 1.0
Madagascar : 1.0
Marshall Islands : 1.0
Martinique : 1.0
Mauritania : 1.0
Mayotte : 1.0
Micronesia (country) : 1.0
Monaco : 1.0
Montserrat : 1.0
New Caledonia : 1.0
Nicaragua : 1.0
Niue : 1.0
North Korea : 1.0
North Macedonia : 1.0
Northern Mariana Islands : 1.0
Palestine : 1.0
Peru : 1.0
Pitcairn : 1.0
Puerto Rico : 1.0
Reunion : 1.0
Saint Barthelemy : 1.0
Saint Helena : 1.0
Saint Kitts and Nevis : 1.0
Saint Lucia : 1.0
Saint Martin (French part) : 1.0
Saint Pierre and Miquelon : 1.0
Saint Vincent and the Grenadines : 1.0
San Marino : 1.0
S

In [750]:
nan_countries = covid_new.groupby('location')['male_smokers'].transform('count') == 0
covid_new.loc[nan_countries, 'male_smokers'] = -1
covid_new['male_smokers'].isna().sum()

0

In [751]:
# handwashing_facilities
for country_name in covid_new['location'].unique():
    country_data = covid_new[covid_new['location'] == country_name]
    ratio_of_null_data = country_data['handwashing_facilities'].isna().sum()/len(country_data)
    if ratio_of_null_data != 0:
        print(f"{country_name} : {ratio_of_null_data}")

Albania : 1.0
American Samoa : 1.0
Andorra : 1.0
Anguilla : 1.0
Antigua and Barbuda : 1.0
Argentina : 1.0
Aruba : 1.0
Australia : 1.0
Austria : 1.0
Bahamas : 1.0
Bahrain : 1.0
Belarus : 1.0
Belgium : 1.0
Bermuda : 1.0
Bonaire Sint Eustatius and Saba : 1.0
Botswana : 1.0
Brazil : 1.0
British Virgin Islands : 1.0
Brunei : 1.0
Bulgaria : 1.0
Canada : 1.0
Cape Verde : 1.0
Cayman Islands : 1.0
Chile : 1.0
China : 1.0
Cook Islands : 1.0


Croatia : 1.0
Curacao : 1.0
Cyprus : 1.0
Czechia : 1.0
Denmark : 1.0
Djibouti : 1.0
Dominica : 1.0
Eritrea : 1.0
Estonia : 1.0
Faeroe Islands : 1.0
Falkland Islands : 1.0
Fiji : 1.0
Finland : 1.0
France : 1.0
French Guiana : 1.0
French Polynesia : 1.0
Gabon : 1.0
Georgia : 1.0
Germany : 1.0
Gibraltar : 1.0
Greece : 1.0
Greenland : 1.0
Grenada : 1.0
Guadeloupe : 1.0
Guam : 1.0
Guernsey : 1.0
Hungary : 1.0
Iceland : 1.0
Iran : 1.0
Ireland : 1.0
Isle of Man : 1.0
Israel : 1.0
Italy : 1.0
Japan : 1.0
Jersey : 1.0
Jordan : 1.0
Kiribati : 1.0
Kosovo : 1.0
Kuwait : 1.0
Latvia : 1.0
Lebanon : 1.0
Libya : 1.0
Liechtenstein : 1.0
Lithuania : 1.0
Luxembourg : 1.0
Malaysia : 1.0
Malta : 1.0
Martinique : 1.0
Mauritius : 1.0
Mayotte : 1.0
Micronesia (country) : 1.0
Monaco : 1.0
Montenegro : 1.0
Montserrat : 1.0
Morocco : 1.0
Nauru : 1.0
Netherlands : 1.0
New Caledonia : 1.0
New Zealand : 1.0
Nicaragua : 1.0
Niue : 1.0
North Korea : 1.0
North Macedonia : 1.0
Northern Mariana Islands : 1.0
Norway : 1.

In [752]:
nan_countries = covid_new.groupby('location')['handwashing_facilities'].transform('count') == 0
covid_new.loc[nan_countries, 'handwashing_facilities'] = -1
covid_new['handwashing_facilities'].isna().sum()

0

In [753]:
# hospital_beds_per_thousand
for country_name in covid_new['location'].unique():
    country_data = covid_new[covid_new['location'] == country_name]
    ratio_of_null_data = country_data['hospital_beds_per_thousand'].isna().sum()/len(country_data)
    if ratio_of_null_data != 0:
        print(f"{country_name} : {ratio_of_null_data}")

American Samoa : 1.0
Andorra : 1.0
Angola : 1.0
Anguilla : 1.0
Aruba : 1.0
Bermuda : 1.0
Bonaire Sint Eustatius and Saba : 1.0
British Virgin Islands : 1.0
Cayman Islands : 1.0
Chad : 1.0
Congo : 1.0


Cook Islands : 1.0
Cote d'Ivoire : 1.0
Curacao : 1.0
Democratic Republic of Congo : 1.0
Faeroe Islands : 1.0
Falkland Islands : 1.0
French Guiana : 1.0
French Polynesia : 1.0
Gibraltar : 1.0
Greenland : 1.0
Guadeloupe : 1.0
Guam : 1.0
Guernsey : 1.0
Guinea-Bissau : 1.0
Isle of Man : 1.0
Jersey : 1.0
Kosovo : 1.0
Lesotho : 1.0
Maldives : 1.0
Martinique : 1.0
Mauritania : 1.0
Mayotte : 1.0
Micronesia (country) : 1.0
Montserrat : 1.0
Namibia : 1.0
New Caledonia : 1.0
Nigeria : 1.0
Niue : 1.0
Northern Mariana Islands : 1.0
Palestine : 1.0
Papua New Guinea : 1.0
Pitcairn : 1.0
Puerto Rico : 1.0
Reunion : 1.0
Rwanda : 1.0
Saint Barthelemy : 1.0
Saint Helena : 1.0
Saint Martin (French part) : 1.0
Saint Pierre and Miquelon : 1.0
Samoa : 1.0
Senegal : 1.0
Sierra Leone : 1.0
Sint Maarten (Dutch part) : 1.0
South Sudan : 1.0
Tokelau : 1.0
Turks and Caicos Islands : 1.0
Tuvalu : 1.0
United States Virgin Islands : 1.0
Vanuatu : 1.0
Vatican : 1.0
Wallis and Futuna : 1.0
Western Sahara : 1.0


In [754]:
nan_countries = covid_new.groupby('location')['hospital_beds_per_thousand'].transform('count') == 0
covid_new.loc[nan_countries, 'hospital_beds_per_thousand'] = -1
covid_new['hospital_beds_per_thousand'].isna().sum()

0

In [755]:
# life_expectancy
for country_name in covid_new['location'].unique():
    country_data = covid_new[covid_new['location'] == country_name]
    ratio_of_null_data = country_data['life_expectancy'].isna().sum()/len(country_data)
    if ratio_of_null_data != 0:
        print(f"{country_name} : {ratio_of_null_data}")

Guernsey : 1.0
Jersey : 1.0
Kosovo : 1.0
Pitcairn : 1.0
Saint Barthelemy : 1.0


In [756]:
nan_countries = covid_new.groupby('location')['life_expectancy'].transform('count') == 0
covid_new.loc[nan_countries, 'life_expectancy'] = -1
covid_new['life_expectancy'].isna().sum()

0

In [757]:
# human_development_index
for country_name in covid_new['location'].unique():
    country_data = covid_new[covid_new['location'] == country_name]
    ratio_of_null_data = country_data['human_development_index'].isna().sum()/len(country_data)
    if ratio_of_null_data != 0:
        print(f"{country_name} : {ratio_of_null_data}")

American Samoa : 1.0
Anguilla : 1.0
Aruba : 1.0
Bermuda : 1.0
Bonaire Sint Eustatius and Saba : 1.0
British Virgin Islands : 1.0
Cayman Islands : 1.0


Cook Islands : 1.0
Curacao : 1.0
Faeroe Islands : 1.0
Falkland Islands : 1.0
French Guiana : 1.0
French Polynesia : 1.0
Gibraltar : 1.0
Greenland : 1.0
Guadeloupe : 1.0
Guam : 1.0
Guernsey : 1.0
Isle of Man : 1.0
Jersey : 1.0
Kosovo : 1.0
Martinique : 1.0
Mayotte : 1.0
Monaco : 1.0
Montserrat : 1.0
Nauru : 1.0
New Caledonia : 1.0
Niue : 1.0
North Korea : 1.0
Northern Mariana Islands : 1.0
Pitcairn : 1.0
Puerto Rico : 1.0
Reunion : 1.0
Saint Barthelemy : 1.0
Saint Helena : 1.0
Saint Martin (French part) : 1.0
Saint Pierre and Miquelon : 1.0
San Marino : 1.0
Sint Maarten (Dutch part) : 1.0
Somalia : 1.0
Tokelau : 1.0
Turks and Caicos Islands : 1.0
Tuvalu : 1.0
United States Virgin Islands : 1.0
Vatican : 1.0
Wallis and Futuna : 1.0
Western Sahara : 1.0


In [758]:
nan_countries = covid_new.groupby('location')['human_development_index'].transform('count') == 0
covid_new.loc[nan_countries, 'human_development_index'] = -1
covid_new['human_development_index'].isna().sum()

0

In [759]:
# total_boosters_smooth
nan_countries = covid_new.groupby('location')['total_boosters'].transform('count') == 0
covid_new.loc[nan_countries, 'total_boosters_smooth'] = -1
smoothed_data = covid_new.groupby('location')['total_boosters'].transform(lambda x: x.interpolate())
covid_new['total_boosters_smooth'] = smoothed_data
covid_new['total_boosters_smooth'].fillna(0, inplace=True)

# Check if there is any null left:
covid_new['total_boosters_smooth'].isna().sum()

0

In [760]:
# total_poeple_fully_vaccinated_smooth
nan_countries = covid_new.groupby('location')['people_fully_vaccinated'].transform('count') == 0
covid_new.loc[nan_countries, 'people_fully_vaccinated_smooth'] = -1
smoothed_data = covid_new.groupby('location')['people_fully_vaccinated'].transform(lambda x: x.interpolate())
covid_new['people_fully_vaccinated_smooth'] = smoothed_data
covid_new['people_fully_vaccinated_smooth'].fillna(0, inplace=True)

# Check if there is any null left:
covid_new['people_fully_vaccinated_smooth'].isna().sum()

0

<div dir=rtl >
<font  size=3>
برای ستون‌های زیر به مقادیر خالی با این منطق پر می‌شود که مقادیر خالی از مقدار پر قبلی پر می‌شود. 
</font>
</div>

In [761]:
# total_vaccinations
nan_countries = covid_new.groupby('location')['total_vaccinations'].transform('count') == 0
covid_new.loc[nan_countries, 'total_vaccinations'] = -1
covid_new['total_vaccinations'] = covid_new.groupby('location')['total_vaccinations'].ffill()
covid_new['total_vaccinations'] = covid_new['total_vaccinations'].fillna(0)
covid_new['total_vaccinations'].isna().sum()

0

In [762]:
# people_vaccinated
nan_countries = covid_new.groupby('location')['people_vaccinated'].transform('count') == 0
covid_new.loc[nan_countries, 'people_vaccinated'] = -1
covid_new['people_vaccinated'] = covid_new.groupby('location')['people_vaccinated'].ffill()
covid_new['people_vaccinated'] = covid_new['people_vaccinated'].fillna(0)
covid_new['people_vaccinated'].isna().sum()

0

In [763]:
# people_fully_vaccinated
nan_countries = covid_new.groupby('location')['people_fully_vaccinated'].transform('count') == 0
covid_new.loc[nan_countries, 'people_fully_vaccinated'] = -1
covid_new['people_fully_vaccinated'] = covid_new.groupby('location')['people_fully_vaccinated'].ffill()
covid_new['people_fully_vaccinated'] = covid_new['people_fully_vaccinated'].fillna(0)
covid_new['people_fully_vaccinated'].isna().sum()

0

In [764]:
# total_boosters
nan_countries = covid_new.groupby('location')['total_boosters'].transform('count') == 0
covid_new.loc[nan_countries, 'total_boosters'] = -1
covid_new['total_boosters'] = covid_new.groupby('location')['total_boosters'].ffill()
covid_new['total_boosters'] = covid_new['total_boosters'].fillna(0)
covid_new['total_boosters'].isna().sum()

0

In [765]:
# new_vaccinations_smoothed
nan_countries = covid_new.groupby('location')['new_vaccinations_smoothed'].transform('count') == 0
covid_new.loc[nan_countries, 'new_vaccinations_smoothed'] = -1
# Other NaN are for days that we didn't have any vaccine so they should be 0.
covid_new['new_vaccinations_smoothed'] = covid_new['new_vaccinations_smoothed'].fillna(0)
covid_new['new_vaccinations_smoothed'].isna().sum()

0

In [766]:
# because we have new_vaccinations_smoothed and the data from new_vaccination has lots of NaN, I wil drop this column.
covid_new = covid_new.drop("new_vaccinations", axis=1)

In [767]:
# new_people_vaccinated_smoothed
nan_countries = covid_new.groupby('location')['new_people_vaccinated_smoothed'].transform('count') == 0
covid_new.loc[nan_countries, 'new_people_vaccinated_smoothed'] = -1
# Other NaN are for days that we didn't have any vaccine so they should be 0.
covid_new['new_people_vaccinated_smoothed'] = covid_new['new_vaccinations_smoothed'].fillna(0)
covid_new['new_people_vaccinated_smoothed'].isna().sum()

0

In [768]:
# total_vaccinations_smooth
covid_new['total_vaccinations_smooth'] = covid_new['new_vaccinations_smoothed'].cumsum()

In [769]:
# total_poeple_vaccinated_smooth
covid_new['total_people_vaccinated_smooth'] = covid_new['new_people_vaccinated_smoothed']

In [770]:
# excess_mortality
nan_countries = covid_new.groupby('location')['excess_mortality'].transform('count') == 0
covid_new.loc[nan_countries, 'excess_mortality'] = -1000
covid_new['excess_mortality'] = covid_new.groupby('location')['excess_mortality'].ffill()
covid_new['excess_mortality'] = covid_new['excess_mortality'].fillna(0)
covid_new['excess_mortality'].isna().sum()

0

In [771]:
covid_new.columns[covid_new.isna().any()].tolist()

[]

In [772]:
covid_new.to_csv('owid-covid-data-clean.csv', index=False)